In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
np.random.seed(123)
torch.manual_seed(123)

### Step 1: Generate Data

In [3]:
from synthetic.generate import SingleTaskTreeDepsGenerator

K = 2
M = 10
N = 10000

# Generate the true class balance to be recovered
class_balance = np.random.random(K)
class_balance /= class_balance.sum()

data = SingleTaskTreeDepsGenerator(N, M, K, class_balance, edge_prob=0.1)
print (f"LF Dependencies: {data.E}")
print (f"Class Balance: {data.p}")

LF Dependencies: []
Class Balance: [0.7087962 0.2912038]


In [4]:
from metal.analysis import lf_summary
lf_summary(data.L,data.Y)

,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
0,"[1, 2]",0.7230,0.7230,0.6359,4475,2755,0.618949
1,"[1, 2]",0.7334,0.7334,0.6460,4688,2646,0.639215
2,"[1, 2]",0.8686,0.8686,0.7530,7363,1323,0.847686
3,"[1, 2]",0.8974,0.8974,0.7746,7975,999,0.888678
4,"[1, 2]",0.7971,0.7971,0.6918,5855,2116,0.734538
5,"[1, 2]",0.8090,0.8090,0.7052,6137,1953,0.758591
6,"[1, 2]",0.8265,0.8265,0.7180,6469,1796,0.782698
7,"[1, 2]",0.8053,0.8053,0.7006,6166,1887,0.765677
8,"[1, 2]",0.9426,0.9426,0.8142,8874,552,0.941439
9,"[1, 2]",0.8265,0.8265,0.7159,6545,1720,0.791894


### Step 2: Learn Dependencies using `DependencyLearnerModel`
**TEMP: hardcoding for now**

In [5]:
edges = data.E
L = np.array(data.L.todense())

### Step 3: Recover the class balance using subset of independent LFs

In [6]:
# Generate indices for independent LFs
ind_lfs = []
for i in range(M):
    if i not in list(sum(edges, ())):
        ind_lfs.append(i)
L_ind = L[:,ind_lfs]

In [7]:
%%time
from metal.label_model.class_balance import ClassBalanceModel

model = ClassBalanceModel(K, abstains=True)
model.train_model(L=L_ind, verbose=False)

print(f"Estimated class balance: {model.class_balance}")
print(f"True class balance: {class_balance}")
print()

Estimated class balance: [0.70349216 0.2962901 ]
True class balance: [0.7087962 0.2912038]

CPU times: user 4.72 s, sys: 96 ms, total: 4.82 s
Wall time: 1.76 s


### Step 4: Train `LabelModel` and predict training labels
**TEMP: not including constraints from Class Balance Model**

In [8]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(k=K, seed=123)

label_model.train_model(data.L, class_balance=model.class_balance.astype(float), deps=edges, n_epochs=500, log_train_every=50)

Computing O...
Estimating \mu...
[50 epo]: TRAIN:[loss=0.017]
[100 epo]: TRAIN:[loss=0.002]
[150 epo]: TRAIN:[loss=0.001]
[200 epo]: TRAIN:[loss=0.001]
[250 epo]: TRAIN:[loss=0.001]
[300 epo]: TRAIN:[loss=0.001]
[350 epo]: TRAIN:[loss=0.001]
[400 epo]: TRAIN:[loss=0.001]
[450 epo]: TRAIN:[loss=0.001]
[500 epo]: TRAIN:[loss=0.001]
Finished Training
CPU times: user 15.6 s, sys: 116 ms, total: 15.7 s
Wall time: 671 ms


In [9]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.982
Precision: 0.986
Recall: 0.989
F1: 0.987
        y=1    y=2   
 l=1   6940    99    
 l=2    77    2884   


### Step 4b: Compare to MajorityVote and No Class Balance+Dependencies LabelModel

**Baseline: No Class Balance or Dependencies**

In [10]:
%%time
from metal.label_model import LabelModel
label_model = LabelModel(k=K, seed=123)

label_model.train_model(data.L, n_epochs=500, log_train_every=50)

Computing O...
Estimating \mu...
[50 epo]: TRAIN:[loss=0.046]
[100 epo]: TRAIN:[loss=0.001]
[150 epo]: TRAIN:[loss=0.001]
[200 epo]: TRAIN:[loss=0.001]
[250 epo]: TRAIN:[loss=0.001]
[300 epo]: TRAIN:[loss=0.001]
[350 epo]: TRAIN:[loss=0.001]
[400 epo]: TRAIN:[loss=0.001]
[450 epo]: TRAIN:[loss=0.001]
[500 epo]: TRAIN:[loss=0.001]
Finished Training
CPU times: user 12.3 s, sys: 136 ms, total: 12.4 s
Wall time: 460 ms


In [11]:
print('Trained Label Model Metrics:')
scores = label_model.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Trained Label Model Metrics:
Accuracy: 0.896
Precision: 0.999
Recall: 0.853
F1: 0.920
        y=1    y=2   
 l=1   5983     7    
 l=2   1034   2976   


**Baseline: Majority Vote**

In [12]:
from metal.label_model.baselines import MajorityLabelVoter

mv = MajorityLabelVoter(seed=123)
print('Majority Label Voter Metrics:')
scores = mv.score((data.L, data.Y), metric=['accuracy','precision', 'recall', 'f1'])

Majority Label Voter Metrics:
Accuracy: 0.964
Precision: 0.984
Recall: 0.965
F1: 0.974
        y=1    y=2   
 l=1   6768    108   
 l=2    249   2875   
